In [1]:
from dask.distributed import Client
import dask.dataframe as dd

import vaex as vx

from pyspark.sql import SparkSession
from pyspark.files import SparkFiles
import pyspark.sql.functions as f


import warnings 
warnings.filterwarnings("ignore")

# Init Dask and Spark

In [2]:
DATA = '/User/yellow_tripdata.csv'
DATAv = 'yellow_tripdata.csv'

In [3]:
DASK_CLIENT = "tcp://mlrun-mydask-3707e08d-4.default-tenant:8786"
client = Client(DASK_CLIENT)

In [4]:
# init spark
spark = SparkSession.builder.appName("Spark job").getOrCreate()

# Read ~80MB csv file

## Vaex

In [5]:
%%time
# vaex will be the fastest only with hdf5 format
# we are using head because it's layz operation 
vdf = vx.read_csv(DATA)
vdf.head(5)

CPU times: user 1.46 s, sys: 189 ms, total: 1.65 s
Wall time: 1.86 s


#,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,01/01/2019 0:46,01/01/2019 0:53,1,1.5,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,nan
1,1,01/01/2019 0:59,01/01/2019 1:18,1,2.6,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,nan
2,2,21/12/2018 13:48,21/12/2018 13:52,3,0,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,nan
3,2,28/11/2018 15:52,28/11/2018 15:55,5,0,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,nan
4,2,28/11/2018 15:56,28/11/2018 15:58,5,0,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,nan


## Dask

In [6]:
%%time
# we are using head because it's layz operation
ddf = dd.read_csv(DATA)
ddf.head(5)

CPU times: user 18.8 ms, sys: 11.3 ms, total: 30.2 ms
Wall time: 1.16 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,01/01/2019 0:46,01/01/2019 0:53,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,01/01/2019 0:59,01/01/2019 1:18,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,21/12/2018 13:48,21/12/2018 13:52,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,28/11/2018 15:52,28/11/2018 15:55,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,28/11/2018 15:56,28/11/2018 15:58,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


## Spark

In [7]:
%%time
# we are using show because it's layz operation
sdf = spark.read.csv(DATAv, sep=",", inferSchema=True, header=True)
sdf.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1|     01/01/2019 0:46|      01/01/2019 0:53|              1|          1.5|         1|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.65|         0.0|                  0.3

# Group by VendorID and Count 

# Vaex

In [8]:
%%time
vdf.groupby(["VendorID"], agg='count')

CPU times: user 17.3 ms, sys: 2.22 ms, total: 19.5 ms
Wall time: 45.7 ms


#,VendorID,count
0,1,378304
1,2,610022
2,4,11673


# Dask

In [9]:
%%time
ddf.groupby(["VendorID"])['total_amount'].count().compute()

CPU times: user 13.8 ms, sys: 3.17 ms, total: 17 ms
Wall time: 1.12 s


VendorID
1    378304
2    610022
4     11673
Name: total_amount, dtype: int64

# Spark

In [11]:
%%time
sdf.groupBy("VendorID").agg({'total_amount': 'count'}).show()

+--------+-------------------+
|VendorID|count(total_amount)|
+--------+-------------------+
|       1|             378304|
|       4|              11673|
|       2|             610022|
+--------+-------------------+

CPU times: user 1.9 ms, sys: 2.22 ms, total: 4.12 ms
Wall time: 1.72 s
